In [44]:
#Let's get started
from __future__ import print_function

import os
import subprocess

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [45]:
#I made this nice csv file because I am not sure how to import the data otherwise
df = pd.read_csv("C:/Users/victo_000/Tree/Logement.csv", index_col=0)

In [46]:
#I don't know why Beds looks odd but other then that, those are our data
df.head(3)

,Baths,Price,Year,Size,PriceperSize,elevation,Location
Beds,,,,,,,
2.0,1.0,999000,1960,1000,999,10,NY
2.0,2.0,2750000,2006,1418,1939,0,NY
2.0,2.0,1350000,1900,2150,628,9,NY


In [47]:
#Let's see how variable 
print("* Number of bath", df["Baths"].unique(), sep="\n")

* Number of bath
[  1.    2.    3.    1.5   5.    4.   10.    7.    2.5   2.3   1.3   3.5
   4.5   6.    5.5   6.5]


In [48]:
#Ok on commence a par arranger un peu tout ca pour préparer l'analyse
def encode_target(df, target_column):
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {Location: n for n, Location in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets)

In [49]:
#Je pense qu'on est presque pret, voyons ce que ca donne
df2, targets = encode_target(df, "Location")
print("* df2.head()", df2[["Target", "Location"]].head(),sep="\n", end="\n\n")
print("* df2.tail()", df2[["Target", "Location"]].tail(),sep="\n", end="\n\n")
print("* targets", targets, sep="\n", end="\n\n")

* df2.head()
      Target Location
Beds                 
2.0        0       NY
2.0        0       NY
2.0        0       NY
1.0        0       NY
0.0        0       NY

* df2.tail()
      Target Location
Beds                 
5.0        1       SF
2.0        1       SF
3.0        1       SF
1.0        1       SF
3.0        1       SF

* targets
['NY' 'SF']



In [40]:
#Et aussi
features = list(df2.columns[:4])
print("* features:", features, sep="\n")

* features:
['Baths', 'Price', 'Year', 'Size']


In [41]:
#On fait notre petit arbre comme ca
y = df2["Target"]
X = df2[features]
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            presort=False, random_state=99, splitter='best')

In [42]:
# Et après
def get_code(tree, feature_names, target_names,
             spacer_base="    "):
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value

    def recurse(left, right, threshold, features, node, depth):
        spacer = spacer_base * depth
        if (threshold[node] != -2):
            print(spacer + "if ( " + features[node] + " <= " + \
                  str(threshold[node]) + " ) {")
            if left[node] != -1:
                    recurse(left, right, threshold, features,
                            left[node], depth+1)
            print(spacer + "}\n" + spacer +"else {")
            if right[node] != -1:
                    recurse(left, right, threshold, features,
                            right[node], depth+1)
            print(spacer + "}")
        else:
            target = value[node]
            for i, v in zip(np.nonzero(target)[1],
                            target[np.nonzero(target)]):
                target_name = target_names[i]
                target_count = int(v)
                print(spacer + "return " + str(target_name) + \
                      " ( " + str(target_count) + " examples )")

    recurse(left, right, threshold, features, 0, 0)

In [43]:
#Résultat (pas joli pcq je peux pas visualiser sous Windows => sad)
get_code(dt, features, targets)

if ( Size <= 804.0 ) {
    if ( Price <= 830000.0 ) {
        if ( Size <= 593.0 ) {
            if ( Year <= 2013.5 ) {
                if ( Year <= 1901.0 ) {
                    return NY ( 3 examples )
                    return SF ( 2 examples )
                }
                else {
                    if ( Year <= 1996.5 ) {
                        return NY ( 26 examples )
                    }
                    else {
                        return NY ( 3 examples )
                        return SF ( 1 examples )
                    }
                }
            }
            else {
                return SF ( 1 examples )
            }
        }
        else {
            if ( Size <= 697.0 ) {
                return NY ( 4 examples )
                return SF ( 15 examples )
            }
            else {
                if ( Year <= 1965.5 ) {
                    if ( Price <= 788500.0 ) {
                        return NY ( 16 examples )
                        re